In [ ]:
from math import e
import pandas as pd
import numpy as np
import cv2 as cv
import random
import matplotlib.pyplot as plt
import random
from xgboost import XGBRegressor
import xgboost as xgb
import pickle
from sklearn.model_selection import GridSearchCV

In [ ]:
test_table= pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')
train_table = pd.read_csv('../input/plant-pathology-2020-fgvc7/train.csv')

### Khám phá dữ liệu

In [ ]:
train_table.head()

- Số dòng, số cột của dữ liệu?

In [ ]:
train_table.shape

- Kiểu dữ liệu ?




In [ ]:
train_table.dtypes

- Có ảnh nào bị trùng không ?

In [ ]:
train_table.index.duplicated().sum()

- Phân bố dữ liệu trong cùng một lớp?

In [ ]:
LABELS = ["healthy", "multiple_diseases", "rust", "scab"]
_, axes = plt.subplots(ncols=4, nrows=1, constrained_layout=True, figsize=(10, 3))
for ax, column in zip(axes, LABELS):
    train_table[column].value_counts().plot.bar(title=column, ax=ax)
plt.show()

- Tỉ lệ dữ liệu giữa các lớp?

In [ ]:
# ADD
plt.title('Label dist')
train_table[LABELS].idxmax(axis=1).value_counts().plot.bar()

## 1. Trích xuất dữ liệu màu sắc và huấn luyện mô hình mạng neuron với 1 layer ẩn và activation là hàm softmax

### 1.1 Trực quan hoá các hình ảnh về bênh tật để đưa ra kết luận

In [ ]:
def getSampleToShow(keyColumn, sample):
    list_sample_image = []
    sample_train = train_table[train_table[keyColumn] == 1].sample(n = sample)
    for image_id in sample_train['image_id']:
        name = "../input/plant-pathology-2020-fgvc7/images/" + image_id + ".jpg"
        image = cv.imread(name)
        im_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        list_sample_image.append(im_rgb)
    return np.array(list_sample_image)

def showImages(images):
    plt.figure(figsize=(15,15))
    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.imshow(images[i])
        
def getImageToTest():
    return cv.imread("../input/plant-pathology-2020-fgvc7/images/Train_382.jpg")

#### Hình ảnh từ sample của class rust

In [ ]:
# Xuat ra anh cua class rust
rust_iamges = getSampleToShow('rust', 9)
showImages(rust_iamges)

#### Hình ảnh từ sample của class scab

In [ ]:
# Xuat ra anh cua class scab
rust_iamges = getSampleToShow('scab', 9)
showImages(rust_iamges)

#### Kết luận:
- Ở đây chúng ta có thể thấy rằng ở class rust thì lá thường sẽ có màu các đốm màu vàng và class scab thì thường sẽ có các đốm màu đen.
- Từ đó vấn đề về màu sắc là một thuộc tính rõ ràng để chúng ta trích xuất dữ liệu cho các class này
- Vậy nên chúng em quyết định chọn 2 figure chính cho model thứ nhất này là color histogram và hue monents hai figure liên quan đến màu sắc.

#### Vấn đề gặp phải:
- Bachground của các ảnh chứa quá nhiều chi tiết về màu sắc thừa cần phải loại bỏ những chi tiết này để mô hình có thể dự đoán tốt hơn.

### 1.2 Tìm ra một số hàm để trích xuất background của ảnh.

#### 1.2.1 Load một ảnh để trích xuất

In [ ]:
image = getImageToTest()
plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))

#### 1.2.2 Sử dụng đạo hàm ảnh bằng phương pháp candy

In [ ]:
def applyCannyThreshold( frame, val):
    ratio = 1.2
    kernel_size = 3
    low_threshold = val
    img_blur = cv.GaussianBlur(frame, (3, 3), 0)
    detected_edges = cv.Canny(img_blur, low_threshold, low_threshold*ratio, kernel_size)
    mask = detected_edges != 0
    dst = frame * (mask[:,:].astype(frame.dtype))
    return dst

gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
drivative_image = applyCannyThreshold(gray_image, 12)
plt.imshow(drivative_image)

In [ ]:
drivative_image.shape

- Sau khi lấy đạo hàm ảnh thì chúng ta có thể thấy các chi tiết của lá được dữ lại khá nhiều với lá ở chính giữa với threshold là 10

#### 1.2.3 Sử dụng phương pháp nén ảnh với threshold và liên kết các ô xung quanh để lấp khuyết ảnh.

In [ ]:
# Hàm dùng để nén ảnh với threshold là ratio sẽ trả ra kết quả là ảnh với block tương ứng với zip x zip
# và nếu block nào không đủ pixel yêu cầu thì là 0 còn ngược lại thì là 1
def zipImage(src, zip_x, zip_y, ratio):
    rs, cs = src.shape
    zip_rs = int(rs / zip_y)
    zip_cs = int(cs / zip_x)
    
    for idx in range(0, zip_rs * zip_y, zip_y):
        for jdx in range(0, zip_cs * zip_x, zip_x):
            block_img = src[idx : idx + zip_y, jdx : jdx + zip_x]
            num_pixel = np.sum(block_img > 0)
            if num_pixel >= zip_x*zip_y * ratio:
                block_img[:,:] = 1
            else:
                block_img[:,:] = 0

    return src

mask_image = zipImage(drivative_image, 8, 8, 0.12)
mask_image = zipImage(mask_image, 16, 16, 0.2)

plt.imshow(mask_image * 255)

- Kết quả của ảnh test sau khi được nén và lấy theo tỉ lệ threshold.

In [ ]:
# Hàm dùng để liên kết các ô xung quanh để lấp khuyết sẽ trả ra kết quả là một ma trận mask
def joinNeiboorPixel(src, zip_x, zip_y, mask_size, ratio):
    rs, cs = src.shape
    zip_rs = int(rs / zip_y)
    zip_cs = int(cs / zip_x)
    half = int(mask_size / 2)
    dst = src.copy()
    for idx in range(half , zip_rs - half):
        for jdx in range(half, zip_cs - half):
            start_row_mask = (idx - half ) * zip_y
            end_row_mask = (idx + half + 1) * zip_y
            start_col_mask = (jdx - half) * zip_x
            end_col_mask = (jdx + half + 1) * zip_x
            mask_block = src[start_row_mask : end_row_mask, start_col_mask : end_col_mask]
            block_img = dst[idx * zip_y : (idx + 1) * zip_y, jdx * zip_x : (jdx + 1) * zip_x]
            num_pixel = np.sum(mask_block > 0)
            if num_pixel >= mask_size * mask_size * zip_x * zip_y * ratio:
                block_img[:,:] = 1
    return dst

mask_image = joinNeiboorPixel(mask_image, 8, 8, 3, 0.2)
mask_image = joinNeiboorPixel(mask_image, 16, 16, 3, 1 / 3)

plt.imshow(mask_image * 255)

- Kết quả của ảnh test sau khi được lấp đầy.

In [ ]:
# Chạy mỗi hàm 2 lần để có một mask nén tốt
# và có sự liên kết giữa các pixel tốt hơn
mask_image = zipImage(drivative_image, 8, 8, 0.12)
mask_image = zipImage(mask_image, 16, 16, 0.2)
mask_image = joinNeiboorPixel(mask_image, 8, 8, 3, 0.2)
mask_image = joinNeiboorPixel(mask_image, 16, 16, 3, 1 / 3)
for chanel in range(0, 3):
    image[:,:,chanel] = image[:,:,chanel] * mask_image

plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))

- Kết quả sau khi loại bỏ bớt background của anh test đã giữ lại gần như chiếc lá gốc.

#### 1.2.4 Hàm dùng để trích xuất color histogram và hue monents

In [ ]:
def fd_histogram(image, mask=None):
    bins = 8
    # convert the image to HSV color-space
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

def fd_hu_moments(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    feature = cv.HuMoments(cv.moments(image)).flatten()
    return feature

#### 1.2.5 Hàm để trích xuất tất cả các ảnh ra tập train_data và lưu lại trong file train_data.csv

In [ ]:
def getPathImageById(image_id):
    return "../input/plant-pathology-2020-fgvc7/images/" + image_id + ".jpg"

def getFigureForImage(path):
    img = cv.imread(path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (3, 3), 0)
    mask_img = applyCannyThreshold(gray, 12)
    mask_img = zipImage(mask_img, 8, 8, 0.12)
    mask_img = zipImage(mask_img, 16, 16, 0.2)
    mask_img = joinNeiboorPixel(mask_img, 8, 8, 3, 0.15)
    mask_img = joinNeiboorPixel(mask_img, 16, 16, 3, 1 / 3)
    for chanel in range(0, 3):
        img[:,:,chanel] = img[:,:,chanel] * mask_img
#     img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    hist_figure = fd_histogram(img).astype(np.float64)
    hu_monents = fd_hu_moments(img)
    fig = np.concatenate((hist_figure, hu_monents))
    return fig

### 1.3 Trích xuất xữ liệu từ dạng ảnh sang dạng số.

In [ ]:
# tạo tên cột cho bảng data mới
def createColsTrainName():
    cols = []
    for i in range(0, 512):
        cols.append('color_hist_' + str(i))

    for i in range(0, 7):
        cols.append('hu_moents_' + str(i))
    
    return cols

# hàm trích xuất data từ ảnh 
def createTrainData(train_table):
    series = train_table['image_id']
    figs = None
    for id in series:
        name = getPathImageById(id)
#         print(name)
        fig = getFigureForImage(name)
        if figs is None:
            figs = fig
        else:
            figs = np.vstack((figs, fig))

#         print("Da trich xuat xong anh: " + name)
    
    cols = createColsTrainName()
    train_data = pd.DataFrame(figs, columns = cols)
    train_data['image_id'] = train_table['image_id']
    return train_data

In [ ]:
# #test
# def getFigureForImage2(path):
#     img = cv.imread(path)
#     gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# #     gray = cv.GaussianBlur(gray, (3, 3), 0)
# #     mask_img = applyCannyThreshold(gray, 12)
# #     mask_img = zipImage(mask_img, 8, 8, 0.12)
# #     mask_img = zipImage(mask_img, 16, 16, 0.2)
# #     mask_img = joinNeiboorPixel(mask_img, 8, 8, 3, 0.15)
# #     mask_img = joinNeiboorPixel(mask_img, 16, 16, 3, 1 / 3)
# #     for chanel in range(0, 3):
# #         img[:,:,chanel] = img[:,:,chanel] * mask_img

# #     hist_figure = fd_histogram(img).astype(np.float64)
# #     hu_monents = fd_hu_moments(img)

# #     fig = np.concatenate((hist_figure, hu_monents))
#     return cv.moments(gray)#hu_monents#fig
# getFigureForImage2("../input/plant-pathology-2020-fgvc7/images/Test_0.jpg")

In [ ]:
# trích xuất data của bảng train và lưu vào train_data.csv
train_data = createTrainData(train_table)
train_data.to_csv('./train_data35.csv', index=False)

In [ ]:
# # Đọc bảng data train để thực hiện train model
# train_data = pd.read_csv('../input/data-plant/train_data2.csv')
# train_data.head()

In [ ]:
# trích xuất data của bảng train và lưu vào test_data.csv
test_data = createTrainData(test_table)
test_data.to_csv('./test_data35.csv', index=False)

In [ ]:
# # Đọc bảng data test_data để thực hiện test model
# test_data = pd.read_csv('../input/data-plant/test_data2.csv')
# test_data.head()

### 1.5 Hiện thực model từ tập dữ liệu đã được trích xuất

#### 1.5.1 Thực hiện lọc lại dữ liệu để train model

In [ ]:
X_train = train_data.drop(columns = ['image_id']).values
X_test = test_data.drop(columns = ['image_id']).values
# X_train

In [ ]:
y=train_table[['healthy', 'multiple_diseases', 'rust', 'scab']]
y.head(2)

In [ ]:
def return_label(_id):
    d=['healthy', 'multiple_diseases', 'rust', 'scab']
    for i in range(4):
        if train_table[d[i]][_id]:
            return i
def y_array(train_table_):
    Y_train_ = train_table_[['healthy', 'multiple_diseases', 'rust', 'scab']]
    Y_train_ =pd.Series(Y_train_.index.values).apply(return_label)
    return Y_train_.values


In [ ]:
Y_train=y_array(train_table)
print(type(Y_train))
Y_train

In [ ]:
# print(X_train.shape)
# print(X_Valid.shape)
# print(Y_Valid.shape)
# print(Y_train.shape)

#### 1.5.2 Thực hiện train model từ tập dữ liệu đã được lọc

In [ ]:
tuned_params = {'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 75}
xgb_model=XGBRegressor(objective='multi:softmax', num_class=4, threads=1,
                       n_estimators=tuned_params['n_estimators'],
                       max_depth=tuned_params['max_depth'],
                       learning_rate=tuned_params['learning_rate'])

xgb_model.fit(X_train, Y_train) 
pickle.dump(xgb_model, open("xgbModel.pkl", "wb"))
print("Saved model to: xgbModel.pkl")

In [ ]:
# #tuning
# def tuning(DO_PARAMETER_TUNING):
#     if DO_PARAMETER_TUNING:
#         parameters = {
#         "n_estimators": [5,25,50,75,100],
#         "max_depth": [2,4,6,9],
#         "learning_rate":  [0.1,0.2, 0.3, 0.4]
#         }
#         xgb_model_ = XGBRegressor(objective='multi:softmax', num_class=4, threads=1)
#         search = GridSearchCV(xgb_model_, parameters, n_jobs=-1, cv=3, verbose=3)
#         search.fit(X_train, Y_train)
#         pickle.dump(search, open("xgbModel_tuning.pkl", "wb"))
#         print("Saved model to: xgbModel_tuning.pkl")
#         tuned_params = search.best_params_
#     else:
#         tuned_params = {'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 75}
#     print(tuned_params)
# tuning(1)
# xgb_model = pickle.load(open("./xgbModel_tuning.pkl", "rb"))

#### 1.5.3 Dự đoán tập test đã được lọc và lưu xuống tập submission

In [ ]:
pred_y = xgb_model.predict(X_train)


In [ ]:
from sklearn.metrics import r2_score
print("Accuracy on training set:", r2_score(Y_train, pred_y))

In [ ]:
pred_y_test = xgb_model.predict(X_test)
pred_y_test.shape

In [ ]:
def submit_(_id):
    d=['image_id','healthy', 'multiple_diseases', 'rust', 'scab']
    l=[submit['image_id'][_id],0,0,0,0]
    
    l[int(pred_y_test[_id])+1]=1
#     print(l)
    return dict(zip(d, l))

In [ ]:
submit=pd.read_csv('../input/plant-pathology-2020-fgvc7/sample_submission.csv')
submit.head(2)

In [ ]:
df=pd.Series(submit.index.values).apply(submit_)
df=pd.DataFrame(list(df))
df.head(2)

In [ ]:
df.to_csv('./submit.csv',index=False)

#### 1.5.4 Nộp kết quả lên trang kagle

#### 1.5.5 Kết luận.
- Kết quả của mô hình vẫn chưa cao do các hệ số trong hàm nén ảnh và lấp ảnh vẫn chưa được phù hợp với 1 số ảnh.
- Do tập dữ liệu khá nhỏ dẫn đến mô hình có thể đã bị trường hợp overfitting.

## 2. Sử dụng CNN với các convolution layer trích xuất dữ liệu và dense layer để huấn luyện dự đoán.

### 2.1 Đọc dữ liệu ảnh vào hai tập train và test

In [ ]:
#   with tf.device('/device:GPU:0'):
#     model.fit(X_train, Y_train, epochs= 40, verbose=1,use_multiprocessing=False, shuffle=True)

#### 2.4 Dự đoán tập test đã được lọc và lưu xuống tập submission

#### 2.5 Nộp kết quả lên trang kagle

#### 2.6 Kết luận về model.
- Kết quả của model CNN tốt hơn hẳn so với model NN.
- Do có nhiều layer convolution nên kết quả dữ liệu được trích xuất tốt hơn là trích xuất thủ công như ở model NN.
- Và là trích xuất trực tiếp từ các phép tích chập nên thông tin được trích xuất chính xác và dẫn đến model đã dự đoán tốt hơn

In [ ]:
#model
# from xgboost import plot_tree
# import matplotlib.pyplot as plt
# plot_tree(xgb_model)
# plt.show()